# Normalisation des données

Afin de permettre le traitement des données sur le cloud Amazon, il nous faut normaliser nos données.

In [ ]:
# Installation des dépendances
%pip install boto3
%pip install smart_open
%pip install python-dotenv
# https://github.com/GeospatialPython/pyshp
%pip install pyshp
%pip install shapely
%pip install matplotlib

## Connexion au bucket S3 Amazon

Pour pouvoir travailler avec nos données, nous utilisons un bucket S3 contenant les dataset. Afin de permettre la connexion au service S3, il est nécessaire d'avoir défini un fichier `.env` contenant les variables d'environnements suivantes :

```
AWS_ACCESS_KEY_ID=xxxxxx
AWS_SECRET_ACCESS_KEY=xxxxxx
```

Ensuite, il suffit d'ouvrir une session à l'aide de la librairie `boto3` :

In [ ]:
import os, boto3, dotenv
from smart_open import open

dotenv.load_dotenv('./.env', override=True)

session = boto3.Session(
    aws_access_key_id = os.environ['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key = os.environ['AWS_SECRET_ACCESS_KEY'],
)

url = 's3://poc-automatic-land-parcel-recognition/test.txt'

with open(url, 'rb', transport_params = {'client': session.client('s3')}) as reader:
    for line in reader:
        print(line.decode('utf8'))


## Fichiers Shapefile

L'étape suivante est d'apprendre à traiter les données parcellaires à notre disposition contenues dans un Shapefile.

Le shapefile, ou « fichier de formes » est un format de fichier pour les systèmes d'informations géographiques (SIG). Initialement développé par ESRI pour ses logiciels commerciaux, ce format est désormais devenu un standard de facto, dont les spécifications sont ouvertes. Nous allons donc procéder à la lecture d'un exemple de fichier SHP contenant le "dallage" global de notre zone à couvrir.

Pour cela, nous utilisons la librairie python `pyshp``:

In [3]:
import shapefile

url = 's3://poc-automatic-land-parcel-recognition/dataset-reference-fr/092/'

myshp = open(url + 'dalles.shp', 'rb', transport_params = {'client': session.client('s3')}) # Shapes
mydbf = open(url + 'dalles.dbf', 'rb', transport_params = {'client': session.client('s3')}) # Records
myprj = open(url + 'dalles.prj', 'rb', transport_params = {'client': session.client('s3')}) # Projection
sr = shapefile.Reader(shp=myshp, dbf=mydbf, prj=myprj)

print(sr.bbox) # Bounding Box

for n in range(0, sr.numRecords):
    print(sr.record(n))
    print(sr.shape(n).points)
    
myshp.close()
mydbf.close()
myprj.close()


[635000.0, 6845000.0, 655000.0, 6875000.0]
Record #0: ['./92-2021-0645-6855-LA93-0M20-E080.jp2']
[(645000.0, 6855000.0), (650000.0, 6855000.0), (650000.0, 6850000.0), (645000.0, 6850000.0), (645000.0, 6855000.0)]
Record #1: ['./92-2021-0640-6870-LA93-0M20-E080.jp2']
[(640000.0, 6870000.0), (645000.0, 6870000.0), (645000.0, 6865000.0), (640000.0, 6865000.0), (640000.0, 6870000.0)]
Record #2: ['./92-2021-0650-6860-LA93-0M20-E080.jp2']
[(650000.0, 6860000.0), (655000.0, 6860000.0), (655000.0, 6855000.0), (650000.0, 6855000.0), (650000.0, 6860000.0)]
Record #3: ['./92-2021-0645-6850-LA93-0M20-E080.jp2']
[(645000.0, 6850000.0), (650000.0, 6850000.0), (650000.0, 6845000.0), (645000.0, 6845000.0), (645000.0, 6850000.0)]
Record #4: ['./92-2021-0645-6860-LA93-0M20-E080.jp2']
[(645000.0, 6860000.0), (650000.0, 6860000.0), (650000.0, 6855000.0), (645000.0, 6855000.0), (645000.0, 6860000.0)]
Record #5: ['./92-2021-0650-6870-LA93-0M20-E080.jp2']
[(650000.0, 6870000.0), (655000.0, 6870000.0), (65500

## Normalisation des données

Notre but maintenant est de normaliser notre dataset. En entrée, nous avons les Shapefiles contenant les géométries des parcelles et des fichiers images JPEG 2000 ou GeoTIFF :

![SampleDataOrthophoto.png](doc/SampleDataOrthophoto.png)

En sortie, on doit produire un fichier JSON dit *manifest* tel que défini dans la [documentation Amazon](https://docs.aws.amazon.com/lookout-for-vision/latest/developer-guide/manifest-files.html) afin d'avoir une données compréhensible par la suite. Ce fichier JSON définit des zones dans l'image, définies par des coordonnées en pixel, qui correspondent aux éléments reconnus dans l'image. Dans notre cas, il s'agira de parcelles.

Nous devons donc mettre en place un processus qui :
- Va lire les fichiers Shapefile
- Va lire les coordonnées des géométries des parcelles (en prenant en compte la projection)
- Va trouver l'image correspondante
- Va repositionner les points en coordonnées locales en pixel dans l'image
- Va générer le [JSON attendu](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-data-output.html#sms-output-video-object-detection)

### Lecture du fichier Shapefile

In [61]:
from shapely.geometry import Polygon, Point, MultiPolygon
import imageio

url = 's3://poc-automatic-land-parcel-recognition/dataset-reference-fr/092/'
limit_dalles = 1
limit_parcels = 3

# Lecture du fichier contenant les dalles
with open(url + 'dalles.shp', 'rb', transport_params = {'client': session.client('s3')}) as dalleshp, open(url + 'dalles.dbf', 'rb', transport_params = {'client': session.client('s3')}) as dalledbf, open(url + 'dalles.prj', 'rb', transport_params = {'client': session.client('s3')}) as dalleprj:
    dallerdr = shapefile.Reader(shp=dalleshp, dbf=dalledbf, prj=dalleprj)

    # Lecture du fichier contenant les parcelles
    with open(url + 'parcelles.shp', 'rb', transport_params = {'client': session.client('s3')}) as parcelleshp, open(url + 'parcelles.dbf', 'rb', transport_params = {'client': session.client('s3')}) as parcelledbf, open(url + 'parcelles.prj', 'rb', transport_params = {'client': session.client('s3')}) as parcelleprj:
        parcelrdr = shapefile.Reader(shp=parcelleshp, dbf=parcelledbf, prj=parcelleprj)

        # On parcours les dalles
        for m in range(0, min(limit_dalles, dallerdr.numRecords)):

            # On fabrique un objet polygone avec les points de notre dalle
            poly = Polygon(dallerdr.shape(m).points)

            # On charge l'image
            # img_blue = Image.open(img_path)

            # On affiche les informations sur la dalle et l'image correspondante
            image = dallerdr.record(m)[0]
            height, width = [ 25000, 25000 ]
            print("Image                                     : " + image)
            print("Format                                    : JPEG 2000")
            print("Dimensions de l'image                     : " + str(width) + "px x " + str(height) + "px")
            print("Système de coordonnées de référence (SCR) : EPSG:2154 - RGF93 v1 / Lambert-93")
            print("Coordonnées des bornes de l'image         : " + str(poly))
            origin = [ poly.boundary.bounds[0], poly.boundary.bounds[1] ]
            print("Origine en bas à gauche (en mètre)        : " + str(origin))
            bounds = [ poly.boundary.bounds[2] - poly.boundary.bounds[0], poly.boundary.bounds[3] - poly.boundary.bounds[1] ]
            print("Dimensions de l'image (en mètre)          : " + str(bounds))

            # Le système de coordonnées Lambert 93 est assez simple : les coordonnées sont exprimées en mètres linéaires
            # à partir d'un origine en bas à gauche du cadre concerné.
            # Voir https://epsg.io/2154
            # Pour déterminer les coordonnées X,Y en pixel des parcelles, on applique pour le moment un produit en croix
            scale = [ width / (poly.boundary.bounds[2] - poly.boundary.bounds[0]), height / (poly.boundary.bounds[3] - poly.boundary.bounds[1]) ]
            print("Echelle (en mètres pour 1px)              : " + str(scale))
            
            # On parcours les parcelles pour trouver celles incluses dans la dalle
            for n in range(0, min(limit_parcels, parcelrdr.numRecords)):
                
                inside = 0
                outside = 0
                parcel = parcelrdr.shape(n)
                for p in parcel.points:
                    if (poly.contains(Point(p))):
                        inside += 1
                    else:
                        outside += 1

                # On ignore les parcelles en dehors ou à cheval entre deux dalles
                if (inside == 0 or outside > 0):
                    continue

                print(" - Parcelle                                        : " + str(parcel))
                print("   Coordonnées 1 en mètres                         : " + str(parcel.points[0]))
                coord = [ parcel.points[0][0] - origin[0], parcel.points[0][1] - origin[1] ]
                print("   Coordonnées 1 en mètres par rapport à l'origine : " + str(coord))
                point = [ coord[0] * scale[0], coord[1] * scale[1] ]
                print("   Coordonnées 1 en pixel par rapport à l'origine  : " + str(point))

                #for p in parcel.points:
                #    coord = [ p[0] - origin[0], p[1] - origin[1] ]
                #    point = [ coord[0] * scale[0], coord[1] * scale[1] ]


Image                                     : ./92-2021-0645-6855-LA93-0M20-E080.jp2
Format                                    : JPEG 2000
Dimensions de l'image                     : 25000px x 25000px
Système de coordonnées de référence (SCR) : EPSG:2154 - RGF93 v1 / Lambert-93
Coordonnées des bornes de l'image         : POLYGON ((645000 6855000, 650000 6855000, 650000 6850000, 645000 6850000, 645000 6855000))
Origine en bas à gauche (en mètre)        : [645000.0, 6850000.0]
Dimensions de l'image (en mètre)          : [5000.0, 5000.0]
Echelle (en mètres pour 1px)              : [5.0, 5.0]
 - Parcelle                                        : Shape #0: POLYGON
   Coordonnées 1 en mètres                         : (649636.4471738542, 6852112.884597713)
   Coordonnées 1 en mètres par rapport à l'origine : [4636.44717385422, 2112.8845977131277]
   Coordonnées 1 en pixel par rapport à l'origine  : [23182.2358692711, 10564.422988565639]
 - Parcelle                                        : Shape 